In [138]:
#Importing Packages
from matplotlib import pyplot as plt
import numpy as np
from scipy.integrate import odeint
import sympy as sy
from scipy import optimize as opt
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd

In [44]:
# ODE Definition Function
def ODE(FM, t, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z):
    P,P_b,P_u,P_e,E,E_b,E_u,S = FM
    dPdt = y - (k*S_e + (k/W)*S)*P + j*P_b + f*(A_e/A_p)*E - e_i*P*(1-P_e/c) + e_o*P_e
    dP_bdt = (k*S_e + (k/W)*S)*P - j*P_b - a*P_b
    dP_udt = a*P_b - g*(A_p/A_e)*P_u
    dP_edt = e_i*P*(1-P_e/c) - e_o*P_e
    dEdt = b*E_u + j*E_b - (k/W)*S*E + g*(A_p/A_e)*P_u - f*(A_e/A_p)*E
    dE_bdt = (k/W)*S*E - j*E_b - a*E_b
    dE_udt = a*E_b - b*E_u - z*E_u
    dSdt = -(k/W)*S*((A_p/V)*P+(A_e/V)*E) + (j+a)*((A_p/V)*P_b+(A_e/V)*E_b) - (v_m*S)/(V*(K_m+S))
    dydt = [dPdt, dP_bdt, dP_udt, dP_edt, dEdt, dE_bdt, dE_udt, dSdt]
    return dydt

In [45]:
# Steady State Functions Creator Function
P, P_b, P_u, P_e, E, E_b, E_u, S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = sy.symbols("P, P_b, P_u, P_e, E, E_b, E_u, S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z")

# Steady State Functions Creator Function
def Steady_State_Functions():
    dPdt = y - (k*S_e + (k/W)*S)*P + j*P_b + f*(A_e/A_p)*E - e_i*P*(1-P_e/c) + e_o*P_e
    dP_bdt = (k*S_e + (k/W)*S)*P - j*P_b - a*P_b
    dP_udt = a*P_b - g*(A_p/A_e)*P_u
    dP_edt = e_i*P*(1-P_e/c) - e_o*P_e
    dEdt = b*E_u + j*E_b - (k/W)*S*E - f*(A_e/A_p)*E + g*(A_p/A_e)*P_u
    dE_bdt = (k/W)*S*E - j*E_b - a*E_b
    dE_udt = a*E_b - b*E_u - z*E_u
    dSdt = -(k/W)*S*((A_p/V)*P+(A_e/V)*E) + (j+a)*((A_p/V)*P_b+(A_e/V)*E_b) - (v_m*S)/(V*(K_m+S))

    sol = sy.solvers.solve([dPdt, dP_bdt, dP_udt, dP_edt, dEdt, dE_bdt, dE_udt],[P, P_b, P_u, P_e, E, E_b, E_u])
    # sol = sy.solvers.solve([dPdt, dP_bdt, dP_udt, dP_edt, dEdt, dE_bdt, dE_udt, dSdt], [P, P_b, P_u, P_e, E, E_b, E_u, S])
    
    return sol

sol = Steady_State_Functions()
# print(sol)

print("Solution with every parameter")
print('    P   = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z:', sol[0][0])
print('    P_b = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z:', sol[0][1])
print('    P_u = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z:', sol[0][2])
print('    P_e = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z:', sol[0][3])
print('    E   = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z:', sol[0][4])
print('    E_b = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z:', sol[0][5])
print('    E_u = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z:', sol[0][6])
print()
print("Solution with only S as a parameter")
print('    P   = lambda S:', sol[0][0])
print('    P_b = lambda S:', sol[0][1])
print('    P_u = lambda S:', sol[0][2])
print('    P_e = lambda S:', sol[0][3])
print('    E   = lambda S:', sol[0][4])
print('    E_b = lambda S:', sol[0][5])
print('    E_u = lambda S:', sol[0][6])


Solution with every parameter
    P   = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z: W*y*(a + j)*(A_e*W*a*b*f + A_e*W*a*f*z + A_e*W*b*f*j + A_e*W*f*j*z + A_p*S*a*k*z)/(A_p*S*a**2*k**2*z*(S + S_e*W))
    P_b = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z: y*(A_e*W*a*b*f + A_e*W*a*f*z + A_e*W*b*f*j + A_e*W*f*j*z + A_p*S*a*k*z)/(A_p*S*a**2*k*z)
    P_u = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z: A_e*y*(A_e*W*a*b*f + A_e*W*a*f*z + A_e*W*b*f*j + A_e*W*f*j*z + A_p*S*a*k*z)/(A_p**2*S*a*g*k*z)
    P_e = lambda S, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z: W*c*e_i*y*(a + j)*(A_e*W*a*b*f + A_e*W*a*f*z + A_e*W*b*f*j + A_e*W*f*j*z + A_p*S*a*k*z)/(A_e*W**2*a**2*b*e_i*f*y + A_e*W**2*a**2*e_i*f*y*z + 2*A_e*W**2*a*b*e_i*f*j*y + 2*A_e*W**2*a*e_i*f*j*y*z + A_e*W**2*b*e_i*f*j**2*y + A_e*W**2*e_i*f*j**2*y*z + A_p*S**2*a**2*c*e_o*k**2*z + A_p*S*S_e*W*a**2*c*e_o*k

In [98]:
# Steady States Solution Function
def Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z, plotting=False):
    """Computes the steady states given parameters
    
    Parameters:
        The values of the parameters in our model
    
    Returns:
        steadies (tuple): The steady states of each state
    """
    # Defining the steady state functions in terms of S
    P   = lambda S: W*y*(a + j)*(A_e*W*a*b*f + A_e*W*a*f*z + A_e*W*b*f*j + A_e*W*f*j*z + A_p*S*a*k*z)/(A_p*S*a**2*k**2*z*(S + S_e*W))
    P_b = lambda S: y*(A_e*W*a*b*f + A_e*W*a*f*z + A_e*W*b*f*j + A_e*W*f*j*z + A_p*S*a*k*z)/(A_p*S*a**2*k*z)
    P_u = lambda S: A_e*y*(A_e*W*a*b*f + A_e*W*a*f*z + A_e*W*b*f*j + A_e*W*f*j*z + A_p*S*a*k*z)/(A_p**2*S*a*g*k*z)
    P_e = lambda S: W*c*e_i*y*(a + j)*(A_e*W*a*b*f + A_e*W*a*f*z + A_e*W*b*f*j + A_e*W*f*j*z + A_p*S*a*k*z)/(A_e*W**2*a**2*b*e_i*f*y + A_e*W**2*a**2*e_i*f*y*z + 2*A_e*W**2*a*b*e_i*f*j*y + 2*A_e*W**2*a*e_i*f*j*y*z + A_e*W**2*b*e_i*f*j**2*y + A_e*W**2*e_i*f*j**2*y*z + A_p*S**2*a**2*c*e_o*k**2*z + A_p*S*S_e*W*a**2*c*e_o*k**2*z + A_p*S*W*a**2*e_i*k*y*z + A_p*S*W*a*e_i*j*k*y*z)
    E   = lambda S: W*y*(a + j)*(b + z)/(S*a*k*z)
    E_b = lambda S: y*(b + z)/(a*z)
    E_u = lambda S: y/z
    
    # Define the dSdt function is terms of the other functions and S
    dSdt = lambda S: -(k/W)*S*((A_p/V)*P(S)+(A_e/V)*E(S)) + (j+a)*((A_p/V)*P_b(S)+(A_e/V)*E_b(S)) - (v_m*S)/(V*(K_m+S))
    # Find the steady state of S using bisection method (the S that makes dSdt=0)
    min_check = 1e-10
    max_check = 1e10
    S_steady = opt.bisect(dSdt, min_check, max_check)

    if plotting:
        S = np.linspace(min_check,max_check,100)
        dSdt_sample = dSdt(S)
        zero = np.zeros_like(S)
        plt.plot(S, dSdt_sample)
        plt.plot(S, zero)
        plt.show()
    
    # Calculate the steady states of the other states
    P_steady = P(S_steady)
    P_b_steady = P_b(S_steady)
    P_u_steady = P_u(S_steady)
    P_e_steady = P_e(S_steady)
    E_steady = E(S_steady)
    E_b_steady = E_b(S_steady)
    E_u_steady = E_u(S_steady)

    # Return the steady states
    return P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady

In [80]:
def P_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return P_steady

In [81]:
def P_b_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return P_b_steady

In [82]:
def P_u_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return P_u_steady

In [83]:
def P_e_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return P_e_steady

In [84]:
def E_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return E_steady

In [85]:
def E_b_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return E_b_steady

In [86]:
def E_u_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return E_u_steady

In [87]:
def S_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return S_steady

In [88]:
def PM_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return P_steady + P_b_steady + P_u_steady + P_e_steady

In [89]:
def EM_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return E_steady + E_b_steady + E_u_steady

In [90]:
def All_steady_state(Params):
    A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Params
    P_steady, P_b_steady, P_u_steady, P_e_steady, E_steady, E_b_steady, E_u_steady, S_steady = Steady_States(A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z)
    return P_steady + P_b_steady + P_u_steady + P_e_steady + E_steady + E_b_steady + E_u_steady

In [58]:
# ODE Solver
def Solve_ODE(ODE, y0, T, num, A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z):
    t_span = np.linspace(0, T, num)
    sol = odeint(ODE, y0, t_span, args = (A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z))
    return t_span, sol

In [70]:
# Standard Parameters Function
def Parameters():
    # Define the parameters
    A_e = 47
    A_p = 314
    a = 1
    b = 1
    c = .15
    e_i = .3
    e_o = .3
    f = .1
    g = .1
    j = 1e2
    K_m = 2.5
    K_d = .74
    k = j/K_d
    S_e = .1
    y = .000083
    W = 32
    v_m = 8.8 * 10**3
    V = 523
    z = .002

    # Return the paramters
    return A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z

In [136]:
# A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z
A_e, A_p, a, b, c, e_i, e_o, f, g, j, K_m, K_d, k, S_e, y, W, v_m, V, z = Parameters()
print(Parameters())
problem = {
    'num_vars': 19,
    'names': ['A_e', 'A_p', 'a', 'b', 'c', 'e_i', 'e_o', 'f', 'g', 'j', 'K_m', 'K_d', 'k', 'S_e', 'y', 'W', 'v_m', 'V', 'z'],
    'bounds': [[10, 100],
               [100, 1000],
               [0.1, 10],
               [0.1, 10],
               [0.01, 1],
               [0.01, 1],
               [0.01, 1],
               [0.01, 1],
               [0.01, 1],
               [10, 1000],
               [0.1, 10],
               [0.1, 10],
               [10, 1000],
               [0.01, 1],
               [1e-5, 1e-3],
               [1, 100],
               [1e3, 1e5],
               [1e2, 1e4],
               [1e-4, 1e-2]]
}
print(len(problem['bounds']))

(47, 314, 1, 1, 0.15, 0.3, 0.3, 0.1, 0.1, 100.0, 2.5, 0.74, 135.13513513513513, 0.1, 8.3e-05, 32, 8800.0, 523, 0.002)
19


In [148]:
param_values = saltelli.sample(problem, 1024)

Y = np.zeros([param_values.shape[0]])

for i, X in enumerate(param_values):
    Y[i] = All_steady_state(X)

/var/folders/8v/664f0v810js5hmb27xj96f040000gn/T/ipykernel_68230/2692843952.py:1: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 1024)


In [151]:
Si = sobol.analyze(problem, Y)
names = np.array(problem['names'])
results = np.vstack([Si['S1'],
                     Si['S1_conf'],
                     Si['ST'],
                     Si['ST_conf']])
print(results.shape)
categories = ["S1", "S1_conf", "ST", "ST_conf"]
results_df = pd.DataFrame(results.T, index=names, columns=categories)
results_interact_df = pd.DataFrame(Si['S2'], index=names, columns=names)
results_df.sort_values(by='ST')

/Users/dgjorgensen/Desktop/MathBio-Research/David_Work/research-env/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


(4, 19)


,S1,S1_conf,ST,ST_conf
K_d,0.000000,0.000000,0.000000,0.000000
V,0.000000,0.000000,0.000000,0.000000
e_i,0.000013,0.000241,0.000008,0.000006
e_o,0.000105,0.000268,0.000013,0.000009
c,-0.000008,0.000165,0.000019,0.000012
g,0.002529,0.004786,0.005829,0.009279
j,-0.000618,0.008000,0.040308,0.043715
b,0.024251,0.022100,0.062393,0.038081
a,0.001708,0.009127,0.065477,0.079282
y,0.023916,0.014491,0.107622,0.041498


In [152]:
results_interact_df

,A_e,A_p,a,b,c,e_i,e_o,f,g,j,K_m,K_d,k,S_e,y,W,v_m,V,z
A_e,NaN,0.003362,-0.000602,0.007503,-0.000536,-0.000529,-0.000576,-0.001534,-0.001630,0.000233,0.006360,-0.000536,0.001702,2.734626e-03,0.001958,-1.915031e-03,2.128336e-03,-0.000536,1.024860e-02
A_p,NaN,NaN,0.009009,0.009322,0.008668,0.008659,0.008652,0.008065,0.009455,0.007741,0.010694,0.008657,0.014483,9.090876e-03,0.007767,8.418610e-03,1.258484e-02,0.008657,7.138664e-03
a,NaN,NaN,NaN,0.010704,0.010045,0.010011,0.010037,0.012219,0.009753,0.009689,0.014634,0.010061,0.010652,1.133875e-02,0.008718,9.155930e-03,1.065474e-02,0.010061,1.456725e-02
b,NaN,NaN,NaN,NaN,-0.004449,-0.004450,-0.004532,-0.008828,-0.004668,-0.003716,-0.008466,-0.004487,0.001827,-2.991675e-03,-0.006756,-2.872196e-03,-5.969919e-03,-0.004487,1.228357e-03
c,NaN,NaN,NaN,NaN,NaN,0.000604,0.000605,0.000136,0.000609,0.000580,0.000645,0.000602,0.000378,4.329173e-04,0.000557,4.464326e-04,6.103647e-04,0.000602,3.907211e-04
e_i,NaN,NaN,NaN,NaN,NaN,NaN,-0.000051,-0.000014,-0.000051,-0.000055,0.000017,-0.000051,0.000008,-5.876412e-05,-0.000062,-1.504783e-05,-5.505305e-05,-0.000051,4.963258e-05
e_o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000141,-0.000148,-0.000124,-0.000189,-0.000141,-0.000120,-2.332794e-04,-0.000136,-7.622509e-05,-7.835623e-05,-0.000141,-1.260282e-04
f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031488,0.036261,0.064321,0.034083,0.018392,2.796008e-02,0.027642,3.937653e-02,4.195193e-02,0.034083,1.164231e-01
g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000386,0.000652,-0.000387,-0.001438,-1.122996e-03,-0.000570,-4.679941e-04,-7.033977e-04,-0.000387,-1.445587e-03
j,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007538,0.003090,0.003318,4.350310e-03,0.001209,1.082874e-02,3.285449e-03,0.003090,1.122284e-02
